## My train using Learner

Training BART with a changed script from the BERT project (adding position_ids)

In [ ]:
!nvidia-smi

In [ ]:
import sys
print(sys.executable)
!which pip

In [ ]:
# !/scratch.ssd/ahajek/tmp/.conda/envs/BARTtrain/bin/python -m pip install pandas

In [ ]:
# !pip install transformers==4.21.0
# !pip install ipywidgets --user
# !pip install pandas==1.4.3
# !pip install wandb
# !pip install rdkit

In [ ]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)


import argparse
from datetime import datetime
import json
import os
import pickle
import random
import time
import wandb
from pathlib import Path
import gc
import glob
from tqdm import tqdm

import torch
import numpy as np
from transformers import TrainingArguments, Trainer, BartConfig, BartForConditionalGeneration
# from transformers.file_utils import logging
# from tensorboardX import SummaryWriter

# custom veci
from data_utils import SpectroDataset, SpectroDataCollator
sys.path.append('data')
sys.path.append('bart_spektro')
from modeling_bart_spektro import BartSpektroForConditionalGeneration
from configuration_bart_spektro import BartSpektroConfig
# from data_preprocess1 import print_args
from bart_spektro_tokenizer import BartSpektroTokenizer
from bart_spektro_trainer import BartSpectroTrainer
from tokenizers import Tokenizer

# from utils import add_special_tokens, generate_sample, sample_seq, set_seed, top_k_top_p_filtering, print_args

In [ ]:
torch.cuda.empty_cache()

#### Setting basic training args

In [ ]:
ncpus=16
ngpus=4 #4
bs = 32 #32 for small model, 16 or 28(slower) for 707M model
gas = 1 # int(16/ngpus)
print("GA:", gas)
which_bart = "spektro" #"original" # "spektro"
data_type = "30M" # "30M"
tokenizer_type = "_bbpe_1M" # for spektro tokenizer use ""


SEQ_LEN = 200
num_epochs = 200 # 10 (BARTy se trenovaly 10 epoch celkem) # int(os.environ["TOTAL_EPOCHS"])
debug_data_len =  None #{"train": 4000, "valid": 4000} # None for no debug   
resume_training = False # True # bool(int(os.environ["RESUME_TRAINING"]))
resume_wandb_id = "" # 2yqhwcas" # "191h62zs" #pass # ""
eval_steps = 10000 # 6500 # 9000 # 30M steps per epoch : 87617 => 9000 steps should be roughly 8hours -> eval_steps, save_steps
save_steps = eval_steps
log_steps = 1 # 5
eval_log_predictions_size = 100 # 100
eval_subset_size = 200000 # 100000 #100k

model = None # aby nebyl nedefinovany

# find the last checkpoint
# checkpoints_pth = "./checkpoints/*"
# runs = sorted(glob.glob(checkpoints_pth))
# checkpoints =  glob.glob(runs[-1]+"/checkpoint-*")
# checkpoints.sort(key=lambda x: int(x.split("-")[-1]))
# load_checkpoint = checkpoints[-1]
# save_checkpoint = runs[-1]
# print(f"last checkpoint: {load_checkpoint}")

# load_checkpoint = "./checkpoints/bart_2022-06-28-10_02_31/checkpoint-18128"

# # LOAD CHECKPOINT FROM
# prefered_run = "./checkpoints/bart_2022-06-28-10_02_31_bigdata/"
# checkpoints =  glob.glob(prefered_run+"checkpoint-*")
# checkpoints.sort(key=lambda x: int(x.split("-")[-1]))
# load_checkpoint = checkpoints[-1]
# print(f"last checkpoint: {load_checkpoint}") # the newest from particular folder

# # SAVE CHECKPOINTS TO  !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# save_checkpoint = "bart_2022-06-28-10_02_31_bigdata/" # if None -> saves to newly generated folder bart_{now}
save_checkpoint = None #"bart_2022-06-28-10_02_31_bigdata" # if None -> saves to newly generated folder bart_{now}

print("save checkpoint to:", save_checkpoint)

#### Setting all training args

In [8]:
# os.environ["PBS_NGPUS"]
!echo $PBS_NGPUS
!echo $CUDA_VISIBLE_DEVICES

1
GPU-0d6541df-b947-e2af-cfe6-d9b6d8fc8d5a


In [9]:
now = str(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
now = now.replace(":","_").replace(" ", "-")
parser = argparse.ArgumentParser()
parser.add_argument("--lr",default=5e-5, type=float, help="learning rate")
parser.add_argument("--seed",default=42, type=int,  help="seed to replicate results")
parser.add_argument("--gradient-accumulation-steps",default=gas, type=int, help="gradient_accumulation_steps")
parser.add_argument("--batch-size",default=bs, type=int,  help="batch_size")
parser.add_argument("--warmup",default=500, type=int,  help="warmup steps for learning rate")
parser.add_argument("--weight-decay",default=0.01, type=float,  help="weight decay rate parameter")
parser.add_argument("--n-gpu",default=ngpus, type=int, required=False, help="no of gpu available")
parser.add_argument("--fairscale",default="", type=str, required=False, choices=["simple", "zero_dp_2", "zero_dp_3"], help="GPU paralellization via Fairscale, " +
                    "more info in HuggingFace's Trainer docs")
parser.add_argument("--deepspeed", default=None, type=str, required=False, help="GPU paralellization via Deepspeed, the value is the location of DeepSpeed json config file; " +
                    "more info in HuggingFace's Trainer docs")
parser.add_argument("--num-workers",default=ncpus, type=int,  help="num of cpus available")
parser.add_argument("--device",default=torch.device('cuda'), help="torch.device object")
parser.add_argument("--num-train-epochs",default=num_epochs, type=int,  help="number of training epochs")
parser.add_argument("--output-dir",default='./output', type=str,  help="Path to save evaluation results")
parser.add_argument("--save-dir",default='./checkpoints', type=str,  help="Path to save trained model")
parser.add_argument("--save-name", type=str, default=f'bart_{now}', help="Name of the model, used for saves")
parser.add_argument("--load-checkpoint", type=str, default='', help="Path to the checkpoint to resume training")
parser.add_argument("--config-dir",default='./configs', type=str,  help="Path to save config files of checkpoints")
parser.add_argument("--fp16",default=True, type=bool, required=False, help="whether to use 16-bit (mixed) precision (through NVIDIA apex) instead of 32-bit")
parser.add_argument("--max-grad-norm",default=1.0, type=float, help="max gradient norm.")
parser.add_argument("--train-data-path",default=f'./data/trial_set/{data_type}{tokenizer_type}_bart_prepared_data_train.pkl', type=str, help="Path to jsonl train dataset")
parser.add_argument("--valid-data-path",default=f'./data/trial_set/{data_type}{tokenizer_type}_bart_prepared_data_valid.pkl', type=str, help="Path to jsonl validation dataset")
parser.add_argument("--log-steps",default=log_steps, type=int,  help="number of steps between logs")
parser.add_argument("--eval-steps",default=eval_steps, type=int,  help="number of steps between evaluations")
parser.add_argument("--tokenizer-path",default='/storage/brno6/home/ahajek/nic', type=str, help="location of the desied tokenizer (special sep token will be added))")
parser.add_argument("--model-path",default='./checkpoints/NECO', type=str, help="location of the desired model to finetune")
parser.add_argument("--wandb", action='store_true', default=True, help="optinal logging via Weights&Biases")
parser.add_argument("--wandb-resume", action='store_true', default=resume_training, help="resume logging via wandb, needs an valid run ID set in args.wandb-id")
parser.add_argument("--wandb-id", type=str, default=wandb.util.generate_id(), help="Process unique wandb ID used for resumin the training process")
parser.add_argument("--tensorboard", action='store_true', default=False, help="optinal logging via TensorBoard")

args = parser.parse_args([])

# extended outputs
# logging.set_verbosity_info()

AttributeError: module 'wandb' has no attribute 'util'

#### Loading tokenizer, data, model

In [10]:
model

In [11]:
# TOKENIZER
tokenizer = Tokenizer.from_file(f"./tokenizer/bbpe_tokenizer/bart{tokenizer_type}_tokenizer.model")
os.environ["TOKENIZERS_PARALLELISM"] = "false" # surpressing a warning

In [12]:
# BART CONIGURATION
if which_bart == "spektro":
    config = BartSpektroConfig(vocab_size = len(tokenizer.get_vocab()),
                                 max_position_embeddings = SEQ_LEN,
                                 max_length = SEQ_LEN,
                                 min_len = 0,
                                 encoder_layers = 12, # 24
                                 encoder_ffn_dim = 4096,
                                 encoder_attention_heads = 16,
                                 decoder_layers = 12,  # 24
                                 decoder_ffn_dim = 4096,
                                 decoder_attention_heads = 16,
                                 encoder_layerdrop = 0.0,
                                 decoder_layerdrop = 0.0,
                                 activation_function = 'gelu',
                                 d_model = 1024,
                                 dropout = 0.2,
                                 attention_dropout = 0.0,
                                 activation_dropout = 0.0,
                                 init_std = 0.02,
                                 classifier_dropout = 0.0,
                                 scale_embedding = False,
                                 use_cache = True,
                                 pad_token_id = 2,
                                 bos_token_id = 3,
                                 eos_token_id = 0,
                                 is_encoder_decoder = True,
                                 decoder_start_token_id = 3,
                                 forced_eos_token_id = 0,
                                 max_log_id=9)

if which_bart == "original":
    config = BartConfig(vocab_size = len(tokenizer.get_vocab()),
                                 max_position_embeddings = SEQ_LEN,
                                 max_length = SEQ_LEN,
                                 min_len = 0,
                                 encoder_layers = 12,
                                 encoder_ffn_dim = 4096,
                                 encoder_attention_heads = 16,
                                 decoder_layers = 12,
                                 decoder_ffn_dim = 4096,
                                 decoder_attention_heads = 16,
                                 encoder_layerdrop = 0.0,
                                 decoder_layerdrop = 0.0,
                                 activation_function = 'gelu',
                                 d_model = 1024,
                                 dropout = 0.2,
                                 attention_dropout = 0.0,
                                 activation_dropout = 0.0,
                                 init_std = 0.02,
                                 classifier_dropout = 0.0,
                                 scale_embedding = False,
                                 use_cache = True,
                                 pad_token_id = 2,
                                 bos_token_id = 3,
                                 eos_token_id = 0,
                                 is_encoder_decoder = True,
                                 decoder_start_token_id = 3,
                                 forced_eos_token_id = 0)

In [13]:
# SAVE NAME
args.save_name = save_checkpoint if save_checkpoint else args.save_name # change args only if user specifies different save name in the initial cell

In [14]:
# DATA
args.train_data_path = f"./data/trial_set/{data_type}_train.pkl"
args.valid_data_path = f"./data/trial_set/{data_type}_valid.pkl"

train_data = SpectroDataset(args.train_data_path, original=which_bart=="original")
valid_data = SpectroDataset(args.valid_data_path, original=which_bart=="original")

if debug_data_len is not None:
    train_data.data = train_data.data[:debug_data_len["train"]]
    valid_data.data = valid_data.data[:debug_data_len["valid"]]

# clean memory
gc.collect()
torch.cuda.empty_cache()

In [15]:
# MODEL
if which_bart == "original":
    model = BartForConditionalGeneration(config)
else:
    model = BartSpektroForConditionalGeneration(config)
# model = BartForConditionalGeneration.from_pretrained(args.model_path)
model.to(args.device)

print("kratkej vypis :D")

kratkej vypis :D


In [16]:
if which_bart == "original":
    try:
        train_data.data.drop(columns=["position_ids"], inplace=True)
        valid_data.data.drop(columns=["position_ids"], inplace=True)
        train_data.data.drop(columns=["decoder_input_ids"], inplace=True)
        valid_data.data.drop(columns=["decoder_input_ids"], inplace=True)
    except:
        pass

In [38]:
print(train_data.data.shape)
train_data.data.head(300)

(5000, 6)


,smiles,input_ids,labels,decoder_attention_mask,encoder_attention_mask,position_ids
16,CCNC(c1cnn2c1OCCC2)CC=C=C,"[26, 27, 28, 30, 38, 39, 40, 41, 42, 43, 44, 4...","[1233, 224, 283, 11, 70, 20, 284, 21, 70, 20, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[3, 6, 5, 5, 5, 9, 8, 9, 9, 7, 9, 8, 6, 3, 7, ..."
19,CCC1CN(C(=O)C)C(CN1C(=O)c1nc2ccccc2cc1Cl)CC,"[36, 38, 39, 40, 41, 42, 43, 44, 49, 50, 51, 5...","[1233, 224, 276, 20, 263, 11, 38, 260, 50, 12,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[3, 0, 6, 5, 7, 8, 8, 6, 3, 1, 5, 4, 6, 6, 5, ..."
50,OC1CNCC1N=c1[nH]c(ncc1[N+](=O)[O-])C(C)(C)C,"[30, 33, 34, 36, 38, 39, 40, 41, 42, 43, 44, 4...","[1233, 224, 286, 20, 400, 20, 49, 32, 70, 20, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[5, 1, 1, 3, 3, 7, 6, 9, 9, 8, 9, 6, 5, 4, 7, ..."
59,COC(=O)Cc1ccc(c(c1)[N+](=O)[O-])NCCc1cnc2n(c1)...,"[33, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 4...","[1233, 224, 285, 260, 50, 12, 279, 20, 280, 11...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[3, 7, 0, 6, 8, 6, 6, 6, 8, 7, 5, 4, 8, 9, 9, ..."
64,O=C(c1ccc2c(c1)n[nH]n2)Nc1ccc(cc1)NS(=O)(=O)c1...,"[33, 34, 36, 37, 38, 39, 40, 41, 42, 43, 44, 4...","[1233, 224, 50, 32, 38, 11, 70, 20, 280, 21, 7...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[3, 2, 2, 1, 3, 7, 5, 6, 4, 6, 6, 6, 0, 2, 4, ..."
...,...,...,...,...,...,...
6796,CC(=CCNCC1(CCOCC1)NC(=O)c1c(C)nsc1C)C,"[33, 39, 40, 41, 42, 43, 44, 45, 51, 52, 53, 5...","[1233, 224, 261, 260, 374, 20, 11, 288, 20, 12...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[5, 8, 7, 9, 8, 9, 8, 6, 7, 7, 8, 6, 8, 8, 8, ..."
6807,CN(C(C(=O)N1CCC1)C)CC1COCCN1,"[15, 18, 27, 28, 32, 38, 39, 40, 41, 42, 43, 4...","[1233, 224, 263, 11, 38, 11, 38, 260, 50, 12, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[2, 0, 1, 3, 1, 1, 5, 5, 8, 8, 8, 8, 6, 4, 1, ..."
6830,COCC1(OC)CCCN(CC1)CCC(=O)C,"[14, 15, 18, 26, 27, 28, 29, 30, 31, 32, 33, 3...","[1233, 224, 306, 20, 11, 286, 12, 299, 11, 261...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 4, 3, 3, 6, 6, 6, 6, 6, 3, 5, 7, 6, 9, 9, ..."
6837,O=C(c1cccc(c1)CNC(=O)c1c(C)ccnc1O)N=c1cc[nH]cc1,"[33, 34, 36, 37, 38, 39, 40, 41, 42, 43, 44, 4...","[1233, 224, 50, 32, 38, 11, 70, 20, 338, 11, 7...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[5, 3, 3, 3, 6, 9, 7, 7, 7, 7, 8, 3, 0, 8, 8, ..."


In [19]:
print(f"Number of model parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")

Number of model parameters: 354206720


#### Setting run resuming and WandB logging

In [20]:
# Resume training
if resume_training:
    args.load_checkpoint = load_checkpoint
    if args.wandb_resume:
        args.wandb_id = resume_wandb_id

# Init wandb
if args.wandb:
    wandb.login()
    run = wandb.init(id=args.wandb_id, resume="allow", entity="hajekad", project="BART_for_gcms")
    wandb.run.name = args.save_name

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: hajekad (msgc_boys). Use `wandb login --relogin` to force relogin
wandb: Currently logged in as: hajekad. Use `wandb login --relogin` to force relogin


#### Setting training arguments (according to args)

In [21]:
training_args = TrainingArguments(
    output_dir=args.save_dir+"/"+args.save_name,         # output directory
    num_train_epochs=args.num_train_epochs,              # total # of training epochs
    weight_decay=args.weight_decay,                                   # strength of weight decay
    logging_dir=args.save_dir + './logs',                # directory for storing logs
    run_name=args.save_name,
)

### custom BartSpectroTrainer arguments
# new things: eval_subset_size, generate_kwargs, eval_log_predictions_size, eval_tokenizer
# change: evaluation_strategy: , eval_steps, logging_steps, save_strategy, save_steps

trainer = BartSpectroTrainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_data,            # training dataset
    eval_dataset=valid_data,             # evaluation dataset
    data_collator = SpectroDataCollator(),
    eval_subset_size = eval_subset_size,
    generate_kwargs = {"num_return_sequences": 1, "top_p": 0.8, "do_sample": True, "num_beams": 1},
    eval_log_predictions_size = eval_log_predictions_size,
    eval_tokenizer = tokenizer,
    w_run = run # wandb run
)

Using cuda_amp half precision backend


### Training

In [22]:
# log args to "args" file in args.save_dir (right before the training starts, to be the latest)
Path(f'{args.save_dir}/{args.save_name}').mkdir(exist_ok=True)
arg_log = print_args(args)
with open(f'{args.save_dir}/{args.save_name}/args', 'w+') as output_file:
    output_file.write(arg_log)

if args.load_checkpoint:
    trainer.train(args.load_checkpoint)
else:
    #trainer.evaluate()
    trainer.train()

arguments:
  lr ........................... 5e-05
  seed ......................... 42
  gradient_accumulation_steps .. 1
  batch_size ................... 32
  warmup ....................... 500
  weight_decay ................. 0.01
  n_gpu ........................ 4
  fairscale .................... 
  deepspeed .................... None
  num_workers .................. 16
  device ....................... cuda
  num_train_epochs ............. 200
  output_dir ................... ./output
  save_dir ..................... ./checkpoints
  save_name .................... bart_2023-04-07-18_17_32
  load_checkpoint .............. 
  config_dir ................... ./configs
  fp16 ......................... True
  max_grad_norm ................ 1.0
  train_data_path .............. ./data/trial_set/DEBUG_train.pkl
  valid_data_path .............. ./data/trial_set/DEBUG_valid.pkl
  log_steps .................... 1
  eval_steps ................... 1000
  tokenizer_path ............... /storage/brno

/scratch.ssd/ahajek/tmp/.local-PyTorch:22.04-py3.SIF/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 5000
  Num Epochs = 200
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 8000
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
/scratch.ssd/ahajek/tmp/.conda/envs/BARTtrain/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, b

Step,Training Loss,Validation Loss



KeyboardInterrupt

